In [43]:
import numpy as np
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import utils

%matplotlib inline

In [3]:
AUDIO_DIR = '/media/ravi/507412DD7412C59E/fma_small/'

# Load metadata and features.
tracks = utils.load('/home/ravi/metafma/fma_metadata/tracks.csv')
genres = utils.load('/home/ravi/metafma/fma_metadata/genres.csv')
features = utils.load('/home/ravi/metafma/fma_metadata/features.csv')
echonest = utils.load('/home/ravi/metafma/fma_metadata/echonest.csv')

tracks.csv


/home/ravi/Amadeus/fma_code/utils.py:216: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  'category', categories=SUBSETS, ordered=True)


genres.csv
features.csv
echonest.csv


In [16]:
filename = utils.get_audio_path(AUDIO_DIR, 10)
print('File: {}'.format(filename))

x = utils.FfmpegLoader().load(filename)
sr=44100 # sampling rate

File: /media/ravi/507412DD7412C59E/fma_small/000/000010.mp3


In [109]:
def id_from_genre_text(genre_text,genres,top_level=True):
    if top_level:
        return genres.loc[genres['title']==genre_text].iloc[0]["top_level"]
    return genres.index[genres['title']==genre_text].tolist()[0]

def get_genre_id(track_id,tracks,genres):
    track_genre_title = tracks[('track','genre_top')].iloc[track_id]
    return id_from_genre_text(track_genre_title,genres)